In [3]:
!ls /data/csv

feature_list.csv      label_eventsV1.2.csv  raw_events.csv
label_eventsV1.1.csv  label_eventsV1.csv    xxxx.csv


In [42]:
!cat /data/jupyter_root/dcube_data/run1-nB.sh

DCUBE_PATH=/data/jupyter_root/dcube_data/
DCUBE_INPUT_PATH=$DCUBE_PATH
DCUBE_OUTPUT_PATH=$DCUBE_PATH
N=$1    # number of input files
D='3' # dimensions of dcube
rm -rf ${DCUBE_PATH}feature*/
for ((i=1; i<=N; i +=5)) 
do
    mkdir -p ${DCUBE_PATH}feature$(($i))
    mkdir -p ${DCUBE_PATH}feature$(($i+1))
    mkdir -p ${DCUBE_PATH}feature$(($i+2))
    mkdir -p ${DCUBE_PATH}feature$(($i+3))
    mkdir -p ${DCUBE_PATH}feature$(($i+4))  
    # read the first line of the input file, and count ','---> dimensions of dcube
    D1=$(head -n 1 ${DCUBE_PATH}/feature$(($i)).txt | grep -o ',' | wc -l)
    D2=$(head -n 1 ${DCUBE_PATH}/feature$(($i+1)).txt | grep -o ',' | wc -l)
    D3=$(head -n 1 ${DCUBE_PATH}/feature$(($i+2)).txt | grep -o ',' | wc -l)
    D4=$(head -n 1 ${DCUBE_PATH}/feature$(($i+3)).txt | grep -o ',' | wc -l)
    D5=$(head -n 1 ${DCUBE_PATH}/feature$(($i+4)).txt | grep -o ',' | wc -l)

    java -cp ./DCube-1.0.jar dcube.Proposed ${DCUBE_PATH}/feature$(($i)).txt   ${DCUBE_PATH}/featu

In [4]:
import time
import pandas as pd
import datetime as dt
dataset = pd.read_csv(open('/data/csv/label_eventsV1.1.csv','r',encoding = 'gb18030'))
dataset.to_csv('/data/csv/label_eventsV1.2.csv',index=True,encoding = 'gb18030')
dataset = pd.read_csv(open('/data/csv/label_eventsV1.2.csv','r',encoding = 'gb18030'))

dataset['time_stamp']=pd.to_datetime(dataset['time_stamp'])#识别为时间格式
dataset['time_stamp_day']=dataset['time_stamp'].dt.strftime('%d')#只保留天格式
dataset['time_stamp_hour']=dataset['time_stamp'].dt.strftime('%d%H')#只保留到小时格式

dic_city={}
i=0
for city in set(dataset['ip_city']):
    dic_city[city]=i
    i+=1
dataset['ip_city']=dataset['ip_city'].map(dic_city)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
def time_to(time,interval):
    start=dt.datetime(2017,10,1,0,0,0)
    while(start<dt.datetime(2017,10,31,23,59,59)):
#         start_str=start.strftime('%d %H')
        if time>=start and time<start+dt.timedelta(hours=interval):
            return start.strftime('%d%H')
        else:
            start+=dt.timedelta(hours=interval)

In [6]:
time1=time.time()
dataset['time_stamp_3hour']=dataset['time_stamp'].apply(lambda x:time_to(x,3))
time2=time.time()
dataset['time_stamp_6hour']=dataset['time_stamp'].apply(lambda x:time_to(x,6))
time3=time.time()
print('3hour:',time2-time1)
print('6hour:',time3-time2)

3hour: 49.26251482963562
6hour: 24.932595014572144


In [30]:
dataset.head(5)

,Unnamed: 0,ip_1,ip_2,ip_3,ip_4,ip,ip_city,email_prefix,email_provider,event_type,...,resource_type,resource_category,label,time_stamp_day,time_stamp_hour,time_stamp_3hour,time_stamp_6hour,ip_12,ip_123,ip_1234
0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,...,NaN,NaN,0,25,2515,2515,2512,NaN,NaN,NaN
1,1,61.0,162.0,94.0,25.0,61.162.94.25,51,NaN,NaN,0,...,NaN,NaN,0,11,1111,1109,1106,61162.0,61162094.0,-1.113433e+09
2,2,112.0,64.0,127.0,42.0,112.64.127.42,33,NaN,NaN,0,...,NaN,NaN,0,30,3011,3009,3006,112064.0,112064127.0,-2.642086e+08
3,3,60.0,16.0,206.0,63.0,60.16.206.63,2,NaN,NaN,0,...,NaN,NaN,0,28,2808,2806,2806,60016.0,60016206.0,-1.139749e+09
4,4,61.0,153.0,148.0,192.0,61.153.148.192,67,NaN,NaN,0,...,NaN,NaN,0,07,0715,0715,0712,61153.0,61153148.0,-1.114008e+09


In [31]:
def float_to_str(x):
    if str(x)=='nan':
        return 'nan'
    else:
        xs=str(int(x))
        while len(xs)<3:
            xs='0'+xs
        return xs
def str_to_float(x):
    if x[0]=='n':
        return np.nan
    else:
        return float(x)
dataset['ip_12']=(dataset['ip_1'].apply(float_to_str)+dataset['ip_2'].apply(float_to_str)).apply(str_to_float)
dataset['ip_123']=(dataset['ip_1'].apply(float_to_str)+dataset['ip_2'].apply(float_to_str)+dataset['ip_3'].apply(float_to_str)).apply(str_to_float)

from IPy import IP
def ip_to_int32(x): 
    if type(x)==str:
        return float(IP(x).int()//2)
dataset['ip_1234']=dataset['ip'].apply(ip_to_int32)
dataset.head()

,Unnamed: 0,ip_1,ip_2,ip_3,ip_4,ip,ip_city,email_prefix,email_provider,event_type,...,resource_type,resource_category,label,time_stamp_day,time_stamp_hour,time_stamp_3hour,time_stamp_6hour,ip_12,ip_123,ip_1234
0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,...,NaN,NaN,0,25,2515,2515,2512,NaN,NaN,NaN
1,1,61.0,162.0,94.0,25.0,61.162.94.25,51,NaN,NaN,0,...,NaN,NaN,0,11,1111,1109,1106,61162.0,61162094.0,517025548.0
2,2,112.0,64.0,127.0,42.0,112.64.127.42,33,NaN,NaN,0,...,NaN,NaN,0,30,3011,3009,3006,112064.0,112064127.0,941637525.0
3,3,60.0,16.0,206.0,63.0,60.16.206.63,2,NaN,NaN,0,...,NaN,NaN,0,28,2808,2806,2806,60016.0,60016206.0,503867167.0
4,4,61.0,153.0,148.0,192.0,61.153.148.192,67,NaN,NaN,0,...,NaN,NaN,0,07,0715,0715,0712,61153.0,61153148.0,516737632.0


In [32]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69521 entries, 0 to 69520
Data columns (total 30 columns):
Unnamed: 0           69521 non-null int64
ip_1                 30676 non-null float64
ip_2                 30676 non-null float64
ip_3                 30676 non-null float64
ip_4                 30676 non-null float64
ip                   30676 non-null object
ip_city              69521 non-null int64
email_prefix         35 non-null float64
email_provider       35 non-null float64
event_type           69521 non-null int64
mobile_prefix_3      243 non-null float64
mobile_city          243 non-null object
time_stamp           69521 non-null datetime64[ns]
user_name            69521 non-null int64
user_agent           69521 non-null int64
os_version           69521 non-null int64
resource_owner       36631 non-null float64
register_type        333 non-null float64
category             3533 non-null float64
status               3533 non-null float64
resource_type        36961 non-n

In [33]:
print(dataset.shape)
dataset_12=dataset[(dataset.event_type==1) | (dataset.event_type==2)]
print(dataset_12.shape)

(69521, 30)
(65325, 30)


In [34]:
device=['user_agent','os_version','ip_1','ip_12','ip_123','ip_1234','ip_city','resource_owner','resource_type','resource_category']
grouplist=[]
    
'''
    #[设备][用户]   用户个数的设备聚集（同一个设备对应多少不同用户）
    grouplist.append([[i],'user_name','nunique'])
    
    #[设备，时间][用户] 用户个数的设备聚集、时间聚集（同一时间同一设备有多少不同用户）
    grouplist.append([[i,'time_stamp'],'user_name','nunique'])
'''

for i in device:
    #[用户][设备]   设备个数的用户聚集（同一个用户对应多少不同设备）
    grouplist.append([['user_name'],i,'nunique'])
    
    # [用户，时间][设备] 用户个数的设备聚集、时间聚集（同一时间同一设备有多少不同用户）
    grouplist.append([['user_name','time_stamp_day'],i,'nunique'])
    grouplist.append([['user_name','time_stamp_hour'],i,'nunique'])
    grouplist.append([['user_name','time_stamp_3hour'],i,'nunique'])
    grouplist.append([['user_name','time_stamp_6hour'],i,'nunique'])

    
#[用户,某个事件类型][事件]  事件的事件类型、用户聚集（同一事件类型，同一用户进行多少次）
grouplist.append([['user_name','event_type'],'Unnamed: 0','count'])

#[用户，时间][总事件数]（时间为周，天）--看用户进行登陆等事件是否具有明显的时间集中性                                                          
grouplist.append([['user_name','time_stamp_day'],'Unnamed: 0','count'])
grouplist.append([['user_name','time_stamp_hour'],'Unnamed: 0','count'])
grouplist.append([['user_name','time_stamp_3hour'],'Unnamed: 0','count'])
grouplist.append([['user_name','time_stamp_6hour'],'Unnamed: 0','count'])

#[用户,设备，时间][总事件]
for i in device:
    grouplist.append([['user_name',i,'time_stamp_day'],'Unnamed: 0','count'])
    grouplist.append([['user_name',i,'time_stamp_hour'],'Unnamed: 0','count'])
    grouplist.append([['user_name',i,'time_stamp_3hour'],'Unnamed: 0','count'])
    grouplist.append([['user_name',i,'time_stamp_6hour'],'Unnamed: 0','count'])
    
#[用户,设备，时间,事件类型][总事件]
for i in device:
    grouplist.append([['user_name',i,'time_stamp_day','event_type'],'Unnamed: 0','count'])
    grouplist.append([['user_name',i,'time_stamp_hour','event_type'],'Unnamed: 0','count'])
    grouplist.append([['user_name',i,'time_stamp_3hour','event_type'],'Unnamed: 0','count'])
    grouplist.append([['user_name',i,'time_stamp_6hour','event_type'],'Unnamed: 0','count'])

user_name_dic=[]
i=1
for x in grouplist:
    print(x[0])
    print(x[1])
    print(x[2])
    save_df=dataset_12.groupby(x[0],as_index=False).agg({x[1]:x[2]})
    for c in range(save_df.shape[1]):
        save_df.iloc[:,c]=save_df.iloc[:,c].apply(int)
    user_name_dic.append(set(save_df['user_name']))
    save_df.to_csv('/data/jupyter_root/dcube_data/feature'+str(i)+'.txt',sep=',',index=None,header=None)
    i+=1

['user_name']
user_agent
nunique
['user_name', 'time_stamp_day']
user_agent
nunique
['user_name', 'time_stamp_hour']
user_agent
nunique
['user_name', 'time_stamp_3hour']
user_agent
nunique
['user_name', 'time_stamp_6hour']
user_agent
nunique
['user_name']
os_version
nunique
['user_name', 'time_stamp_day']
os_version
nunique
['user_name', 'time_stamp_hour']
os_version
nunique
['user_name', 'time_stamp_3hour']
os_version
nunique
['user_name', 'time_stamp_6hour']
os_version
nunique
['user_name']
ip_1
nunique
['user_name', 'time_stamp_day']
ip_1
nunique
['user_name', 'time_stamp_hour']
ip_1
nunique
['user_name', 'time_stamp_3hour']
ip_1
nunique
['user_name', 'time_stamp_6hour']
ip_1
nunique
['user_name']
ip_12
nunique
['user_name', 'time_stamp_day']
ip_12
nunique
['user_name', 'time_stamp_hour']
ip_12
nunique
['user_name', 'time_stamp_3hour']
ip_12
nunique
['user_name', 'time_stamp_6hour']
ip_12
nunique
['user_name']
ip_123
nunique
['user_name', 'time_stamp_day']
ip_123
nunique
['user_name

In [35]:
!ls /data/jupyter_root/dcube_data/

8-4-10.log	feature12.txt	feature30      feature50.txt  feature71
8-4-174.log	feature13	feature30.txt  feature51      feature71.txt
8-5-1.log	feature130.txt	feature31      feature51.txt  feature72
8-5-2.log	feature131.txt	feature31.txt  feature52      feature72.txt
clean.sh	feature132.txt	feature32      feature52.txt  feature73
DCube-1.0.jar	feature133.txt	feature32.txt  feature53      feature73.txt
DCube-2.0.jar	feature134.txt	feature33      feature53.txt  feature74.txt
Density.txt	feature135.txt	feature33.txt  feature54      feature75.txt
evaluate.txt	feature13.txt	feature34      feature54.txt  feature76.txt
feature1	feature14	feature34.txt  feature55      feature77.txt
feature10	feature14.txt	feature35      feature55.txt  feature78.txt
feature100.txt	feature15	feature35.txt  feature56      feature79.txt
feature101.txt	feature15.txt	feature36      feature56.txt  feature7.txt
feature102.txt	feature16	feature36.txt  feature57      feature8
feature103.txt	feature16.txt	feature37      fea

In [36]:
Feature_combination=[]
Standard=[]
Poly=[]
for group in grouplist:
    Feature_combination.append(group[0])
    Standard.append(group[1])
    Poly.append(group[2])
lis_name=[]
for i in range(1,len(grouplist)+1):
    lis_name.append('feature'+str(i))
feature_list=pd.DataFrame({'Feature_combination':Feature_combination,'Standard':Standard,'Poly':Poly,'user_name_dic':user_name_dic},index=lis_name)
feature_list['user_name_num']=feature_list['user_name_dic'].apply(len)
feature_list.to_csv('/data/csv/feature_list.csv')
feature_list_mini=pd.DataFrame({'Feature_combination':Feature_combination,'Standard':Standard,'Poly':Poly},index=lis_name)
feature_list_mini['user_name_num']=feature_list['user_name_num']
feature_list_mini.to_csv('/data/jupyter_root/feature_list.csv')
feature_list

,Feature_combination,Standard,Poly,user_name_dic,user_name_num
feature1,[user_name],user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1...",9409
feature2,"[user_name, time_stamp_day]",user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1...",9409
feature3,"[user_name, time_stamp_hour]",user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1...",9409
feature4,"[user_name, time_stamp_3hour]",user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1...",9409
feature5,"[user_name, time_stamp_6hour]",user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1...",9409
feature6,[user_name],os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1...",9409
feature7,"[user_name, time_stamp_day]",os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1...",9409
feature8,"[user_name, time_stamp_hour]",os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1...",9409
feature9,"[user_name, time_stamp_3hour]",os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1...",9409
feature10,"[user_name, time_stamp_6hour]",os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1...",9409


# 测试

In [37]:
feature_list.to_csv('/data/jupyter_root/GSD/feature_list.csv')

In [171]:
feature_list.loc[['feature83','feature86']]

,Feature_combination,Standard,Poly,user_name_dic,user_name_num
feature83,"[user_name, ip_1234, time_stamp_day]",Unnamed: 0,count,"{458762, 1310734, 1343510, 1540121, 1671194, 1...",7106
feature86,"[user_name, ip_1234, time_stamp_6hour]",Unnamed: 0,count,"{458762, 1310734, 1343510, 1540121, 1671194, 1...",7106


In [148]:
    x=grouplist[135]
    print(x[0])
    print(x[1])
    print(x[2])
    save_df=dataset_12.groupby(x[0],as_index=False).agg({x[1]:x[2]})
    for c in range(save_df.shape[1]):
        save_df.iloc[:,c]=save_df.iloc[:,c].apply(int)
    save_df.to_csv('/data/jupyter_root/GSD/test1.txt',sep=',',index=None,header=None)
    

['user_name', 'ip_1234', 'time_stamp_hour', 'event_type']
Unnamed: 0
count
